In [9]:
from bpemb import BPEmb
from tqdm import tqdm
import csv
import numpy as np
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
bpemb_en=BPEmb(lang="en",dim=50)

In [3]:
print(bpemb_en.encode("Stratford"))
print(bpemb_en.embed("Stratford").shape)

['▁strat', 'ford']
(2, 50)


In [24]:
with open("datasets/Chatbot/train.csv") as f:
    reader = csv.reader(f, delimiter='\t')
    max_len = 0
    y = []
    for row in tqdm(reader):
        y.append(row[1])
        sample_len = len(bpemb_en.encode(row[0]))
        max_len = sample_len if sample_len > max_len else max_len


100it [00:00, 25075.05it/s]


In [25]:
print(max_len)
print(y[:10])

32
['FindConnection', 'FindConnection', 'FindConnection', 'FindConnection', 'FindConnection', 'FindConnection', 'FindConnection', 'FindConnection', 'FindConnection', 'FindConnection']


In [26]:
max_len = max_len + 1

In [27]:
# label encoder
le = LabelEncoder()
encoded_labels = le.fit_transform(y)
#print(encoded_labels)
print(le.classes_)

['DepartureTime' 'FindConnection']


In [28]:
y = to_categorical(encoded_labels, num_classes=len(le.classes_))

In [29]:
x = None

In [30]:
with open("datasets/Chatbot/train.csv") as f:
    reader = csv.reader(f, delimiter='\t')
    for row in tqdm(reader):
        embeddings = bpemb_en.embed(row[0])
        padding_vec = np.zeros((max_len - embeddings.shape[0], 50))
        padded = np.vstack((embeddings, padding_vec))
        padded = np.expand_dims(padded, axis=0)
        #print(padded.shape)
        if x is not None:
            x = np.vstack((x, padded))
        else:
            x = padded

100it [00:00, 3130.38it/s]


In [31]:
x.shape

(100, 33, 50)

In [34]:
batch_size = 32
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 2

In [35]:
print("Bulding the Neural Network")

Bulding the Neural Network


In [33]:
model = Sequential()

# Dropping 20% nodes from the embedding layers
model.add(Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(le.classes_))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))